In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
from numpy import cov
import math

TRAIN = "COL341_SVM_data/DHC_train.csv"
TEST = "COL341_SVM_data/DHC_test.csv"
TEST_LABELS = "COL341_SVM_data/orig_DHC_target_labels.txt"

def LinearKernel(x1, x2,sigma=None):
    return np.dot(x1, x2)

def GaussianKernel(x, y, sigma=1.0):
    return (math.exp(sigma*((linalg.norm(x-y))**2)))


In [2]:
class SVM(object):
    def __init__(self,kernel,soft,gamma=0.01):
        self.kernel=kernel
        self.soft=soft
        self.gamma = -1*gamma

    def fit(self, X, y):
        self.X_tr = X
        self.y_tr = y
        self.n,self.m=X.shape
        Gram = np.array([[self.kernel(i,j,self.gamma) for j in X] for i in X])
        print (np.sum(Gram))

        P = cvxopt.matrix(np.outer(y,y) * Gram,(self.n,self.n),'d')
        q = cvxopt.matrix(np.ones(self.n) * -1)
        A = cvxopt.matrix(y, (1,self.n),'d')
        b = cvxopt.matrix(0.0)

        g1 = np.diag(np.ones(self.n) * -1)
        g2 = np.identity(self.n)
        G = cvxopt.matrix(np.vstack((g1, g2)))
        H1 = np.zeros(self.n)
        H2 = np.ones(self.n) * self.soft
        h = cvxopt.matrix(np.hstack((H1, H2)))

        model=cvxopt.solvers.qp(P, q, G, h, A, b)

        self.multipliers = np.ravel(model['x'])
        print (self.multipliers)
        self.SV = np.array([x for x,y in enumerate(self.multipliers) if y > epslon])

        self.points = self.multipliers[self.SV]
        self.X = X[self.SV]
        self.Y = y[self.SV]

        self.weights=np.zeros(self.m)
        for i in range(len(self.points)):
            self.weights += self.points[i]*self.X[i]*self.Y[i]

        self.bias=0
        for i in range(len(self.points)):
            self.bias -= np.dot(self.weights,self.X[i]) - self.Y[i]
        self.bias= self.bias/self.SV.shape[0]

        return True

    def project(self,x):
        xn = np.array([np.sum((x-xtr)**2) for xtr in self.X_tr])
        return (np.dot((self.points*self.y_tr).transpose(),np.exp(self.gamma*xn)))
    
    def predict(self,data):
        if(self.kernel == LinearKernel):
            return np.sign(np.dot(data, self.weights) + self.bias)
        
        y_pred = []
        for i in data:
            y = self.project(i)
            print (y)
            if y[0] > 0:
                y_pred.append(1)
            else:
                y_pred.append(-1)

        return np.asarray(y_pred)

In [3]:
tr_dat = pd.read_csv(TRAIN,header=None).values
ts_dat = pd.read_csv(TEST,header=None).values
X_tr = tr_dat[:,1:]/255
X_ts = ts_dat[:,1:]/255
y_tr = np.sign(tr_dat[:,:1] - 0.5)
y_ts = np.sign(np.genfromtxt(TEST_LABELS) - 0.5)

In [11]:
## BEST PARAMS ###
SOFT = 10
model = SVM(LinearKernel,SOFT)
epslon = 0.00000001
model.fit(X_tr,y_tr)
y_pred = model.predict(X_ts)
print ("Accuracy = ", float(np.sum(y_ts==y_pred))/y_ts.shape[0])
## BEST PARAMS ###


1010575492.5304109
     pcost       dcost       gap    pres   dres
 0: -2.1498e+03 -3.1557e+05  1e+06  1e+00  1e-11
 1: -8.5371e+02 -1.3413e+05  2e+05  2e-01  8e-12
 2:  1.7609e+02 -3.6215e+04  6e+04  5e-02  6e-12
 3:  3.1972e+02 -1.6730e+04  3e+04  2e-02  4e-12
 4:  3.6775e+02 -7.3095e+03  1e+04  6e-03  3e-12
 5:  2.1721e+02 -1.6422e+03  2e+03  7e-04  2e-12
 6:  1.8368e+01 -2.2145e+02  2e+02  4e-05  2e-12
 7: -2.0150e+01 -1.3748e+02  1e+02  1e-05  1e-12
 8: -3.2804e+01 -1.2960e+02  1e+02  4e-06  1e-12
 9: -4.8619e+01 -8.7716e+01  4e+01  9e-07  1e-12
10: -5.5506e+01 -7.3372e+01  2e+01  1e-07  1e-12
11: -5.9268e+01 -6.5925e+01  7e+00  4e-08  1e-12
12: -6.1018e+01 -6.2580e+01  2e+00  3e-09  1e-12
13: -6.1536e+01 -6.1770e+01  2e-01  2e-15  1e-12
14: -6.1634e+01 -6.1646e+01  1e-02  6e-15  1e-12
15: -6.1639e+01 -6.1640e+01  2e-04  9e-15  1e-12
16: -6.1639e+01 -6.1639e+01  3e-06  1e-14  1e-12
Optimal solution found.
[6.42337230e-11 3.07057270e-10 1.41985781e-10 ... 9.80182975e-11
 5.95073768

In [5]:
# SOFT = 1
# gamma = 0.01
# epslon = 0.00001
# print ("building Model")
# model = SVM(GaussianKernel,SOFT,gamma=gamma)
# model.fit(X_tr,y_tr)
# print ("Done training")
# y_pred = model.predict(X_ts)
# print ("Accuracy = ", float(np.sum(y_ts==y_pred))/y_ts.shape[0])

In [6]:
x_cov = cov(tr_dat[:,1:].T)
u, lmb, v = linalg.svd(x_cov)
X_tr_pca = np.dot(tr_dat[:,1:],u[:,:50])/255
X_ts_pca = np.dot(ts_dat[:,1:],u[:,:50])/255

In [10]:
## BEST PARAMS ###
SOFT = 0.01
model = SVM(LinearKernel,SOFT)
epslon = 0.00000001
model.fit(X_tr_pca,y_tr)
y_pred = model.predict(X_ts_pca)
print ("Accuracy = ", float(np.sum(y_ts==y_pred))/y_ts.shape[0])
# BEST PARAMS ###

981166251.4899012
     pcost       dcost       gap    pres   dres
 0: -3.7939e+02 -7.5701e+01  3e+04  2e+02  1e-12
 1: -1.1015e+01 -7.5124e+01  5e+02  3e+00  1e-12
 2: -5.8733e+00 -5.2951e+01  1e+02  4e-01  1e-13
 3: -4.0620e+00 -2.0546e+01  2e+01  7e-02  3e-14
 4: -3.7829e+00 -7.8213e+00  5e+00  1e-02  2e-14
 5: -4.1193e+00 -5.9828e+00  2e+00  4e-03  2e-14
 6: -4.3176e+00 -5.2952e+00  1e+00  2e-03  2e-14
 7: -4.4122e+00 -5.0013e+00  6e-01  9e-04  2e-14
 8: -4.4819e+00 -4.8403e+00  4e-01  5e-04  2e-14
 9: -4.5349e+00 -4.7197e+00  2e-01  2e-04  2e-14
10: -4.5762e+00 -4.6445e+00  7e-02  4e-05  2e-14
11: -4.5961e+00 -4.6158e+00  2e-02  6e-06  2e-14
12: -4.6023e+00 -4.6080e+00  6e-03  1e-06  2e-14
13: -4.6045e+00 -4.6053e+00  8e-04  1e-16  2e-14
14: -4.6049e+00 -4.6050e+00  8e-05  1e-16  2e-14
15: -4.6049e+00 -4.6049e+00  1e-06  2e-16  2e-14
Optimal solution found.
[8.00592792e-11 4.49108502e-09 3.56348446e-10 ... 9.99999958e-03
 6.41425131e-11 4.31622683e-10]
Accuracy =  0.97


In [8]:
SOFT = 1
gamma = 0.01
epslon = 0.00000001
print ("building Model")
model = SVM(GaussianKernel,SOFT,gamma=gamma)
model.fit(X_tr_pca,y_tr)
print ("Done training")
y_pred = model.predict(X_ts_pca)
print ("Accuracy = ", float(np.sum(y_ts==y_pred))/y_ts.shape[0])

building Model
2561170.084862799
     pcost       dcost       gap    pres   dres
 0: -2.5448e+02 -5.8238e+03  3e+04  2e+00  3e-15
 1: -1.5838e+02 -3.0704e+03  4e+03  2e-01  4e-15
 2: -1.5298e+02 -6.5526e+02  6e+02  2e-02  3e-15
 3: -2.0434e+02 -3.8468e+02  2e+02  6e-03  3e-15
 4: -2.2880e+02 -3.1331e+02  9e+01  2e-03  3e-15
 5: -2.4500e+02 -2.7232e+02  3e+01  2e-04  4e-15
 6: -2.5158e+02 -2.5929e+02  8e+00  4e-14  4e-15
 7: -2.5376e+02 -2.5574e+02  2e+00  3e-15  4e-15
 8: -2.5441e+02 -2.5474e+02  3e-01  1e-14  4e-15
 9: -2.5454e+02 -2.5455e+02  9e-03  2e-14  4e-15
10: -2.5455e+02 -2.5455e+02  1e-04  1e-14  4e-15
Optimal solution found.
[1.92396643e-08 4.83641963e-01 4.79634398e-08 ... 6.40532569e-02
 1.96058247e-08 6.91746132e-08]
Done training
[9.05417039e-07 2.27601515e+01 2.25715558e-06 ... 3.01434107e+00
 9.22648516e-07 3.25535168e-06]
[1.32766047e-06 3.33744033e+01 3.30978555e-06 ... 4.42008632e+00
 1.35292789e-06 4.77349281e-06]
[-2.07693109e-06 -5.22093845e+01 -5.17767659e-06 ..

[-6.47003447e-07 -1.62642140e+01 -1.61294451e-06 ... -2.15402293e+00
 -6.59316916e-07 -2.32624710e-06]
[-1.85417767e-06 -4.66098635e+01 -4.62236438e-06 ... -6.17298288e+00
 -1.88946552e-06 -6.66654162e-06]
[2.37281892e-06 5.96473402e+01 5.91530891e-06 ... 7.89965862e+00
 2.41797731e-06 8.53127314e-06]
[-1.10897346e-06 -2.78771030e+01 -2.76461071e-06 ... -3.69202711e+00
 -1.13007893e-06 -3.98722188e-06]
[-1.66703327e-06 -4.19054737e+01 -4.15582356e-06 ... -5.54993626e+00
 -1.69875948e-06 -5.99367952e-06]
[-2.34898907e-06 -5.90483112e+01 -5.85590237e-06 ... -7.82032357e+00
 -2.39369394e-06 -8.44559488e-06]
[1.35050612e-06 3.39486918e+01 3.36673853e-06 ... 4.49614475e+00
 1.37620833e-06 4.85563247e-06]
[-4.93247478e-07 -1.23991341e+01 -1.22963922e-06 ... -1.64213403e+00
 -5.02634735e-07 -1.77343029e-06]
[1.53823233e-06 3.86677070e+01 3.83472978e-06 ... 5.12112835e+00
 1.56750725e-06 5.53058643e-06]
[1.62340421e-06 4.08087369e+01 4.04705866e-06 ... 5.40468508e+00
 1.65430008e-06 5.83681484

[-9.88327818e-07 -2.48443423e+01 -2.46384766e-06 ... -3.29037007e+00
 -1.00713721e-06 -3.55345049e-06]
[2.07731591e-06 5.22190579e+01 5.17863591e-06 ... 6.91586128e+00
 2.11685042e-06 7.46881662e-06]
[1.94635619e-06 4.89270246e+01 4.85216044e-06 ... 6.47986633e+00
 1.98339834e-06 6.99796184e-06]
[-1.72874842e-06 -4.34568539e+01 -4.30967607e-06 ... -5.75540015e+00
 -1.76164916e-06 -6.21557121e-06]
[-1.43850846e-06 -3.61608729e+01 -3.58612358e-06 ... -4.78912472e+00
 -1.46588548e-06 -5.17203755e-06]
[1.90709961e-06 4.79402025e+01 4.75429593e-06 ... 6.34917219e+00
 1.94339465e-06 6.85681809e-06]
[-9.70530679e-07 -2.43969622e+01 -2.41948036e-06 ... -3.23111931e+00
 -9.89001369e-07 -3.48946236e-06]
[-2.00783171e-06 -5.04723812e+01 -5.00541558e-06 ... -6.68453245e+00
 -2.04604383e-06 -7.21899197e-06]
[6.16738375e-07 1.55034180e+01 1.53749533e-06 ... 2.05326356e+00
 6.28475854e-07 2.21743155e-06]
[3.43381944e-07 8.63185111e+00 8.56032567e-07 ... 1.14319728e+00
 3.49917030e-07 1.23460123e-06]


[-2.10751124e-06 -5.29781007e+01 -5.25391123e-06 ... -7.01638846e+00
 -2.14762041e-06 -7.57738142e-06]
[-1.37836293e-06 -3.46489494e+01 -3.43618405e-06 ... -4.58888646e+00
 -1.40459530e-06 -4.95578932e-06]
[1.69830955e-06 4.26916891e+01 4.23379363e-06 ... 5.65406217e+00
 1.73063099e-06 6.10613079e-06]
[1.28946591e-06 3.24142777e+01 3.21456858e-06 ... 4.29292786e+00
 1.31400642e-06 4.63616745e-06]
[-1.61944428e-07 -4.07091932e+00 -4.03718677e-07 ... -5.39150158e-01
 -1.65026479e-07 -5.82257725e-07]
[-2.55755235e-06 -6.42911238e+01 -6.37583933e-06 ... -8.51467859e+00
 -2.60622650e-06 -9.19546682e-06]
[1.09077908e-06 2.74197370e+01 2.71925309e-06 ... 3.63145382e+00
 1.11153828e-06 3.92180548e-06]
[4.33450530e-07 1.08959731e+01 1.08056867e-06 ... 1.44305627e+00
 4.41699759e-07 1.55843534e-06]
[2.33685928e-06 5.87433956e+01 5.82566347e-06 ... 7.77994073e+00
 2.38133331e-06 8.40198324e-06]
[-1.54893519e-06 -3.89367529e+01 -3.86141142e-06 ... -5.15676063e+00
 -1.57841380e-06 -5.56906767e-06]


[1.83708549e-06 4.61802045e+01 4.57975450e-06 ... 6.11607909e+00
 1.87204804e-06 6.60508811e-06]
[1.99036514e-06 5.00333108e+01 4.96187237e-06 ... 6.62638223e+00
 2.02824485e-06 7.15619236e-06]
[-1.81591880e-07 -4.56481215e+00 -4.52698709e-07 ... -6.04561033e-01
 -1.85047852e-07 -6.52898505e-07]
[-1.88554174e-06 -4.73982860e+01 -4.70055332e-06 ... -6.27740109e+00
 -1.92142650e-06 -6.77930856e-06]
[-2.49247547e-07 -6.26552371e+00 -6.21360618e-07 ... -8.29802271e-01
 -2.53991110e-07 -8.96148829e-07]
[4.89714328e-07 1.23103187e+01 1.22083127e-06 ... 1.63037136e+00
 4.99034344e-07 1.76072715e-06]
[1.42854124e-06 3.59103195e+01 3.56127586e-06 ... 4.75594157e+00
 1.45572858e-06 5.13620126e-06]
[1.07405578e-06 2.69993509e+01 2.67756282e-06 ... 3.57577813e+00
 1.09449671e-06 3.86167825e-06]
[1.92173263e-06 4.83080436e+01 4.79077524e-06 ... 6.39788886e+00
 1.95830615e-06 6.90942988e-06]
[8.16253436e-07 2.05187787e+01 2.03487555e-06 ... 2.71749497e+00
 8.31787993e-07 2.93477136e-06]
[-1.87618841

[2.60529061e-07 6.54911564e+00 6.49484821e-07 ... 8.67361020e-01
 2.65487329e-07 9.36710575e-07]
[2.42162778e-06 6.08742854e+01 6.03698675e-06 ... 8.06215452e+00
 2.46771508e-06 8.70676134e-06]
[-1.86866957e-06 -4.69741574e+01 -4.65849191e-06 ... -6.22122975e+00
 -1.90423322e-06 -6.71864606e-06]
[-9.41210102e-08 -2.36599087e+00 -2.34638575e-07 ... -3.13350438e-01
 -9.59122776e-08 -3.38404266e-07]
[-2.30130494e-06 -5.78496394e+01 -5.73702844e-06 ... -7.66157218e+00
 -2.34510231e-06 -8.27415058e-06]
[1.03697135e-06 2.60671314e+01 2.58511334e-06 ... 3.45231554e+00
 1.05670651e-06 3.72834426e-06]
[-2.70941725e-06 -6.81086664e+01 -6.75443028e-06 ... -9.02027168e+00
 -2.76098162e-06 -9.74148443e-06]
[5.46705109e-07 1.37429390e+01 1.36290619e-06 ... 1.82010675e+00
 5.57109747e-07 1.96563276e-06]
[1.14433637e-06 2.87660470e+01 2.85276851e-06 ... 3.80975833e+00
 1.16611485e-06 4.11436626e-06]
[1.30561096e-06 3.28201282e+01 3.25481733e-06 ... 4.34667846e+00
 1.33045874e-06 4.69421565e-06]
[-1.40

[1.09580024e-08 2.75459578e-01 2.73177059e-08 ... 3.64817044e-02
 1.11665501e-08 3.93985866e-08]
[1.59621942e-06 4.01253723e+01 3.97928846e-06 ... 5.31418069e+00
 1.62659793e-06 5.73907420e-06]
[1.77687346e-06 4.46666095e+01 4.42964920e-06 ... 5.91561946e+00
 1.81069010e-06 6.38860080e-06]
[2.96406655e-06 7.45099780e+01 7.38925718e-06 ... 9.86805762e+00
 3.02047729e-06 1.06570548e-05]
[5.23418188e-07 1.31575581e+01 1.30485316e-06 ... 1.74257924e+00
 5.33379640e-07 1.88190657e-06]
[-1.04155815e-06 -2.61824333e+01 -2.59654798e-06 ... -3.46758604e+00
 -1.06138060e-06 -3.74483571e-06]
[1.57494809e-06 3.95906590e+01 3.92626019e-06 ... 5.24336356e+00
 1.60492177e-06 5.66259491e-06]
[-1.76372099e-06 -4.43359857e+01 -4.39686078e-06 ... -5.87183184e+00
 -1.79728731e-06 -6.34131216e-06]
[-1.79161916e-06 -4.50372831e+01 -4.46640941e-06 ... -5.96471125e+00
 -1.82571642e-06 -6.44161771e-06]
[-1.47588983e-06 -3.71005566e+01 -3.67931331e-06 ... -4.91357588e+00
 -1.50397828e-06 -5.30643916e-06]
[-2.72

[2.07969008e-06 5.22787393e+01 5.18455460e-06 ... 6.92376545e+00
 2.11926978e-06 7.47735276e-06]
[3.31652855e-07 8.33700814e+00 8.26792583e-07 ... 1.10414845e+00
 3.37964718e-07 1.19243026e-06]
[3.70799453e-09 9.32106572e-02 9.24382929e-09 ... 1.23447646e-02
 3.77856336e-09 1.33317860e-08]
[-2.74939379e-06 -6.91135870e+01 -6.85408964e-06 ... -9.15336277e+00
 -2.80171898e-06 -9.88521678e-06]
[2.50177855e-06 6.28890958e+01 6.23679828e-06 ... 8.32899482e+00
 2.54939124e-06 8.99493677e-06]
[2.45566176e-06 6.17298231e+01 6.12183161e-06 ... 8.17546143e+00
 2.50239678e-06 8.82912767e-06]
[-2.89153427e-06 -7.26866795e+01 -7.20843815e-06 ... -9.62658104e+00
 -2.94656461e-06 -1.03962711e-05]
[-6.89906876e-07 -1.73427099e+01 -1.71990043e-06 ... -2.29685829e+00
 -7.03036864e-07 -2.48050281e-06]
[1.50368206e-06 3.77991908e+01 3.74859783e-06 ... 5.00610257e+00
 1.53229943e-06 5.40636380e-06]
[-1.32152776e-07 -3.32202408e+00 -3.29449704e-07 ... -4.39966913e-01
 -1.34667846e-07 -4.75144317e-07]
[-8.95

In [9]:
# print (y_pred)
print (model.project(X_ts_pca[1]))

[1.32766047e-06 3.33744033e+01 3.30978555e-06 ... 4.42008632e+00
 1.35292789e-06 4.77349281e-06]
